## MinHash at work

The libray 'datasketch' gives you probabilistic data structures that can process and search very large
amount of data super fast, with little loss of accuracy.

datasketch.MinHash lets you estimate the Jaccard similarity
between sets of arbitrary sizes in linear time, O(m+n), using a small and fixed memory space.
It can also be used to compute Jaccard similarity between data streams


In [1]:
from datasketch.minhash import MinHash
from datasketch.lsh import MinHashLSH

"""
Datasketch uses shingles made of one word.
"""

document_1 = 'minhash is a probabilistic data structure for'
document_1 += ' estimating the similarity between datasets'

document_2 = 'minhash is a probability data structure for'
document_2 += ' estimating the similarity between documents'

document_3 = 'This is a completly different probabilistic thing'

ModuleNotFoundError: No module named 'datasketch'

In [ ]:
# split creates a list from a string using white spaces as separator
# the function set converts lists to sets

use_shingles = False

if use_shingles:
    shing_len = 5
    set_1 = set([document_1[i:i + shing_len] for i in range(len(document_1) - shing_len + 1)])
    set_2 = set([document_2[i:i + shing_len] for i in range(len(document_2) - shing_len + 1)])
    set_3 = set([document_3[i:i + shing_len] for i in range(len(document_3) - shing_len + 1)])
else:
    set_1 = set(document_1.split())
    set_2 = set(document_2.split())
    set_3 = set(document_3.split())
  

In [ ]:
  
# compute and print Jaccard similarity between set_1 and set_2
jd = len(set_1.intersection(set_2)) / len(set_1.union(set_2))
print("Jaccard similarity between set_1 and set_2 is: %.2f" % jd)

# compute and print Jaccard similarity between set_1 and set_3
jd = len(set_1.intersection(set_3)) / len(set_1.union(set_3))
print("Jaccard similarity between set_1 and set_3 is: %.2f" % jd)

# compute and print Jaccard similarity between set_2 and set_3
jd = len(set_2.intersection(set_3)) / len(set_2.union(set_3))
print("Jaccard similarity between set_2 and set_3 is: %.2f" % jd)

In [ ]:
num_perm = 32 # try different permutations

# Costruct the min hash of our reference document
m1 = MinHash(num_perm=num_perm) # construct the object MinHash
for s in set_1:
    m1.update(s.encode('utf8')) # let's populate m1

a = m1.digest() 
# extract into an numpy array the MinHash actual value
print(a)

In [ ]:
# Costruct the min hash of the other document to asses similarity
m2 = MinHash(num_perm=num_perm) # new object
m3 = MinHash(num_perm=num_perm) # new object
# feed the min hashes with data of document 2 and 3
for s in set_2:
    m2.update(s.encode('utf8')) # populate
for s in set_3:
    m3.update(s.encode('utf8')) # populate

print("Jaccard similarity between min hashes of set_1 and set_2 is: %.2f" % m1.jaccard(m2))
print("Jaccard similarity between min hashes of set_1 and set_3 is: %.2f" % m1.jaccard(m3))
print("Jaccard similarity between min hashes of set_2 and set_3 is: %.2f" % m2.jaccard(m3))

In [ ]:
# Create LSH index
threshold = 0.6  # Jaccard similarity
lsh = MinHashLSH(threshold=threshold, num_perm=num_perm) # construct the object LSH
lsh.insert("m2", m2) # let's populate
lsh.insert("m3", m3) # let's populate
#lsh.insert("m1", m1) # let's populate

result = lsh.query(m1) # is m1 similar to m2 or m3
print("Approximate neighbours of m1 with Jaccard similarity >= %f" % threshold, result)


# excercise

# add 3 or more string documents with various level of similarities and rerun the query. Observation.
# add 3 or more documents with lot of typo errors and rerun similarity query. Observations.
# try with different shingles size.
# try with different numbers of permutation. Observation.
# print the difference vector of jaccard based on set and jaccard based on MinHashes for various permutations
# Is mx.jaccard(my) equal to my.jaccard(mx)? Try them.

In [ ]:
# MinHash merging

document = """In computer science and data mining, MinHash 
(or the min-wise independent permutations locality sensitive hashing scheme) is a technique for quickly 
estimating how similar two sets are. The scheme was invented by Andrei Broder (1997),[1] and initially 
used in the AltaVista search engine to detect duplicate web pages and eliminate them from search results.
It has also been applied in large-scale clustering problems, such as clustering documents by the similarity 
of their sets of words"""

ds = document.split()
half_low = ds[0:int(len(ds)/2)]
half_up = ds[int(len(ds)/2):]

half_up


In [ ]:
mh_doc = MinHash(num_perm=num_perm) # construct the object MinHash
for s in ds:
    mh_doc.update(s.encode('utf8'))


mh_low = MinHash(num_perm=num_perm)
for s in half_low:
    mh_low.update(s.encode('utf8'))


mh_up = MinHash(num_perm=num_perm)
for s in half_up:
    mh_up.update(s.encode('utf8'))


print("Jaccard similarity between min hashes of doc and half_low is: %.2f" % mh_doc.jaccard(mh_low))
print("Jaccard similarity between min hashes of doc and half_up is: %.2f" % mh_doc.jaccard(mh_up))

# excercise.
# compute jaccard distance based on sets

In [ ]:
mh_low.merge(mh_up)
print("Jaccard similarity between min hashes of doc and merge of low and up is: %.2f" % mh_low.jaccard(mh_doc))